# Group Module

The Group module allows you to combine several elements of a pandapower net into a group. Various functions are available, which are then automatically applied to all elements in this group.
This tutorial shows you how to create groups and how to use some helpful simple group functions.

To analyse the group functionality we use the CIGRE MV net.

In [ ]:
import numpy as np
from pandapower.networks import create_cigre_network_mv
from pandapower.create import create_group_from_dict, create_group
from pandapower.groups import (
    set_group_in_service,
    set_group_reference_column,
    group_element_index,
    set_value_to_group,
    set_group_out_of_service,
    group_res_q_mvar,
    group_res_p_mw,
    attach_to_group,
    detach_from_group,
    count_group_elements,
    remove_not_existing_group_members,
    ensure_lists_in_group_element_column
)
from pandapower.toolbox.grid_modification import get_connected_elements_dict
from pandapower.run import runpp

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
net = create_cigre_network_mv(with_der="all")
net.switch["closed"] = True

## Create Groups
As examples, we define two groups, one to represent a virtual power plant with sgens, loads and a storage and for all elements of the second feeder of the net.
You can create groups using lists of element types and element indices, or you can pass all that information in one dict.

In [ ]:
# define a Group as virtual power plant
gr1_name = "virtual power plant"
vpp_element_types = ["storage", "sgen", "load"]
vpp_elements = [[1], [6, 8, 9, 10, 11, 12], [5, 6]]
gr1_idx = create_group(net, vpp_element_types, vpp_elements, name=gr1_name)

# define a Group of a Feeder 2
gr2_name = "Feeder2"
feeder2buses = [12, 13, 14]
feeder2_elements_dict = get_connected_elements_dict(net, feeder2buses)
feeder2_elements_dict["bus"] = feeder2buses
gr2_idx = create_group_from_dict(net, feeder2_elements_dict, name=gr2_name)

Now we can see that there are entries in `net.group` for both groups. As usual in pandapower, the information is accessed by means of the index. For groups with multiple element types, e.g. the virtual power plant includes storages, sgens and loads, multiple rows with the same index are created to `net.group`.

In [ ]:
print(net.group)

## Set `reference_column`
However, the user can manipulate the indices of the net element dataframes and some helper functions change/may not preserve the element indices, such as `create_continuous_elements_index(net)`. As a result, by means of the indices, a group can no longer find its members.
For that reason, groups can also detect their members by a column of the elements dataframes. That can be applied directly at the group definition or later using `set_group_reference_column()`. Then we can see, that the group does no longer store the indices but the values of the set reference column:

In [ ]:
print("Group 1 data with indices:\n")
print(net.group.loc[gr1_idx])

set_group_reference_column(net, gr1_idx, "name")

print("\nAfter setting 'name' as reference column, the group stores the members by the names:\n")
print(net.group.loc[gr1_idx])

As we can see, the second group now stores the names of the members and not indices. Using `group_element_index()` you can nevertheless get the indices of the group members. You can see that these are still the same as given in the definition of the group above.

In [ ]:
print(group_element_index(net, gr1_idx, "load"))

**Attention:** Be aware that `reference_column` only works fine if there are no duplicated values in `net[element][reference_column]` for all members of the groups!

## Set Values to all Group Members
The following code block shows you how to set the value to all members of a group. A specific use case of the is to set all members in service or out of service. For that reason, these got explicit function names.

In [ ]:
# setting a name value to group 2 members
set_value_to_group(net, gr2_idx, "member of group '%s'" % gr2_name, "name")

# visualize the effect
print("The load names:\n")
print(net.load.name)

# set all elements of group 2 out of service
set_group_out_of_service(net, gr2_idx)
runpp(net)
print("\nThe bus results with Feeder 2 out of service:\n")
print(net.res_bus)  # the Feeder 2 buses should now have nan values

# and back in service...
set_group_in_service(net, gr2_idx)
runpp(net)
print("\nThe bus results with Feeder 2 back in service:\n")
print(net.res_bus)

## Sum Group Consumption Power
Groups can sum its complete power consumption including losses. Since the virtual power plant group predominantly consists of generation units, its active power value is negative.

In [ ]:
for gr_idx, gr_name in zip([gr1_idx, gr2_idx], [gr1_name, gr2_name]):
    print("Group '%s' consumes %.2f MW and %.2f Mvar." % (
        gr_name, group_res_p_mw(net, gr_idx), group_res_q_mvar(net, gr_idx)))

# a validation of Feeder 2 group values is easy since there is only one trafo and one line
# which supply the feeder:
p_val = net.res_line.p_to_mw.at[14] + net.res_trafo.p_hv_mw.at[1]
assert np.isclose(group_res_p_mw(net, gr2_idx), p_val)

## Append and Drop Group Members
Once defined, it still easy to change the members of a group:

In [ ]:
p_before = group_res_p_mw(net, gr1_idx)

# append the virtual power plant
attach_to_group(net, gr1_idx, ["storage"], [[net.storage.name.at[0]]], "name")
# drop an sgen from the virtual power plant
detach_from_group(net, gr1_idx, "load", [5])

# validate via compare the active power consumption
assert np.isclose(p_before + net.res_storage.p_mw.at[0] - net.res_load.p_mw.at[5], group_res_p_mw(net, gr1_idx))

## Some more functionality

For small groups, `print(net.group.loc[[group_index]]` can be used to see quickly how many members are included. For larger groups, `count_group_elements()` is available to access the number of group members per element type:

In [ ]:
no_member = count_group_elements(net, gr2_idx)
print("Number of member per element type:")
print(no_member)
print(f"\n Overall number of members: {no_member.sum()}")

If `net.group` has been corrupted, `ensure_lists_in_group_element_column()` and `remove_not_existing_group_members()` may help:

In [ ]:
# assume net.group is corrupted for some reason, as...
net.group.element_index.iat[-1] = 5  # no list
net.trafo.drop(1, inplace=True)  # net elements were dropped without using toolbox functions like drop_trafos()

ensure_lists_in_group_element_column(net)  # fixes no list data
remove_not_existing_group_members(net)  # detects that transformer 1 does not exist and drops it
print(net.group)

To explore some more functions, such as `groups_equal()`, `compare_group_elements()` and others, please have a look to the Code.